In [15]:
from langchain_community.tools import WikipediaQueryRun,ArxivQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage,AIMessage
import os   
import dotenv


from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END, START
from langgraph.types import Send
from langchain_core.messages import HumanMessage, SystemMessage
import operator


In [2]:
dotenv.load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [3]:
from langchain_classic.agents import initialize_agent,AgentType
llm = ChatGroq(model = "llama-3.3-70b-versatile",verbose=True,max_tokens=500,api_key=GROQ_API_KEY)

In [16]:
class OverallState(TypedDict):
    task: str
    subtasks: list[str]
    results: Annotated[list[str], operator.add]
    final_output: str

In [17]:
class WorkerState(TypedDict):
    subtask: str
    result: str

In [18]:
def orchestrator(state: OverallState):
    task = state["task"]
    
    messages = [
        SystemMessage(content="You are an orchestrator. Break down the task into 3 independent subtasks that can be done in parallel."),
        HumanMessage(content=f"Break down this task: {task}\n\nReturn ONLY a numbered list of 3 subtasks, nothing else.")
    ]
    response = llm.invoke(messages)
    
    subtasks = [line.strip() for line in response.content.split('\n') if line.strip() and any(char.isdigit() for char in line[:3])]
    subtasks = [task.split('.', 1)[1].strip() if '.' in task else task for task in subtasks]
    
    return {"subtasks": subtasks}


In [ ]:
def continue_to_workers(state: OverallState):
    return [Send("worker", {"subtask": task}) for task in state["subtasks"]]


def worker(state: WorkerState):
    subtask = state["subtask"]
    
    messages = [
        SystemMessage(content="You are a specialized worker. Complete the given subtask thoroughly."),
        HumanMessage(content=f"Complete this subtask: {subtask}")
    ]
    response = llm.invoke(messages)
    
    return {"results": [f"Subtask: {subtask}\nResult: {response.content}"]}

In [26]:
def aggregator(state: OverallState):
    results = state["results"]
    
    combined_results = "\n\n".join(results)
    
    messages = [
        SystemMessage(content="You are an aggregator. Combine the results from all workers into a cohesive final output."),
        HumanMessage(content=f"Original task: {state['task']}\n\nWorker results:\n{combined_results}\n\nProvide a final summary.")
    ]
    response = llm.invoke(messages)
    
    return {"final_output": response.content}


In [27]:
def create_orchestrator_workflow():
    workflow = StateGraph(OverallState)
    
    workflow.add_node("orchestrator", orchestrator)
    workflow.add_node("worker", worker)
    workflow.add_node("aggregator", aggregator)
    
    workflow.add_edge(START, "orchestrator")
    workflow.add_conditional_edges("orchestrator", continue_to_workers, ["worker"])
    workflow.add_edge("worker", "aggregator")
    workflow.add_edge("aggregator", END)
    
    return workflow.compile()

In [33]:
app = create_orchestrator_workflow()

initial_state = {
    "task": "Plan a 3-day trip to Tokyo including food, attractions, and accommodation",
    "subtasks": [],
    "results": [],
    "final_output": ""
}

In [34]:
# Just one call
final_state = app.invoke(initial_state)

# Access everything from final_state
print(final_state["subtasks"])
print(final_state["results"])
print(final_state["final_output"])

['Research and book accommodation options in Tokyo for the 3-day trip', 'Create an itinerary of top attractions and experiences to visit in Tokyo', 'Plan a food itinerary, including restaurants, cafes, and street food to try during the trip']
["Subtask: Research and book accommodation options in Tokyo for the 3-day trip\nResult: **Researching Accommodation Options in Tokyo**\n\nTokyo offers a wide range of accommodation options to suit different budgets and preferences. Here are some options to consider for the 3-day trip:\n\n**Luxury Hotels**\n\n1. **Park Hyatt Tokyo**: Located in the heart of Shinjuku, this 5-star hotel offers stunning views of the city and luxurious amenities. (Approx. $500 per night)\n2. **The Ritz-Carlton, Tokyo**: Situated in the Roppongi district, this hotel features elegant rooms and a world-class spa. (Approx. $450 per night)\n3. **Four Seasons Hotel Tokyo at Marunouchi**: This 5-star hotel is located in the financial district and offers sleek, modern rooms an